# Cost Analysis

## 1. Import Library

In [ ]:
import pandas as pd
import numpy as np
import pyodbc # SQL Connection
import sqlCredentials as sql

## 2. Connect to Databases

In [ ]:
#PROCUREMENTDB

proc_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

#BookXCenterProduction
prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

## 3. Define Clean up tool

In [ ]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

## 4. Define Tables

In [ ]:
# Dictionary
pub_dict = """
SELECT *
FROM dbo.PublisherDictionary
"""
# LP All currencies
all_lp = """
SELECT
	[Isbn]
	,[Currency]
	,[Price]
FROM
	[Isbn].[ListPrice]
"""
# Bibliography
bilblo_sql = """
SELECT Isbn
    , Title
    , Publisher
    , Author
FROM 
    Isbn.Bibliography
"""
# Echange Rate
exchange_rate = """
SELECT 
    name,
    rate
FROM dbo.XChange
"""
#Supplier Mega List
megalist_sql = """
SELECT 
    [ISBN]
    ,[Supplier]
    ,[Publisher]
    ,[Currency]
    ,[ListPrice]
    ,[Discount]
    ,[CostUnitPrice]
    ,[MaxQtyPerOrder]
    ,[ShipmentOrigin]
    ,[UnitShippingCost]
    ,[SeaFreightCost] AS sea_shipping_cost
  FROM 
    [Process].[SupplierMegaList]
"""

#Import supplier Excel
#Import short_discount
#Import restrictions

### 4.1 Import Dictionary and clean it

In [ ]:
pub_dict = pd.read_sql(pub_dict, proc_db)
pub_dict.head()

In [ ]:
pub_dict.columns = map(str.lower, pub_dict.columns)
pub_dict = pub_dict.fillna('N/A')
for col in list(pub_dict.columns):
    pub_dict[col] = pub_dict.apply(lambda x: clean_up(x[col]), axis =1)

pub_dict.head()

### 4.2 Import Bibliography and clean it

In [ ]:
# IMport and read biblio
biblio = pd.read_sql(bilblo_sql, prod_db)
biblio.head()

In [ ]:
biblio.columns = map(str.lower, biblio.columns)
biblio = biblio.fillna('N/A')
biblio['isbn'] = biblio['isbn'].astype(str)
for col in list(biblio.columns):
    biblio[col] = biblio.apply(lambda x: clean_up(x[col]), axis =1)
biblio.head()

Now we Merge bibliography with the dictionary

In [ ]:
biblio2 = pd.merge(biblio, pub_dict, how='left', left_on = 'publisher', right_on = 'publisherlong' )
biblio2.drop(columns = ['publisherlong'], inplace = True)
biblio2.rename(columns={'publishershort': 'pub'}, inplace = True)
biblio2.head()

### 4.3 Import SupplierMegaList and clean it

In [ ]:
megalist = pd.read_sql(megalist_sql,proc_db)
megalist.head()

In [ ]:
megalist.columns = map(str.lower, megalist.columns)
megalist = megalist.fillna('N/A')
megalist['isbn'] = megalist['isbn'].astype(str)
for col in ['isbn', 'supplier', 'publisher', 'currency', 'shipmentorigin']:
    megalist[col] = megalist.apply(lambda x: clean_up(x[col]), axis =1)
megalist.head()

In [ ]:
megalist_pub = pd.merge(megalist, biblio2, how='left', on = 'isbn' )
megalist_pub.head()

In [ ]:
#cleaning pub name
megalist_pub['pub'] = np.where(
    megalist_pub['pub'] == 'N/A'
    , megalist_pub['publisher_x']
    , megalist_pub['pub'])
    
# centralizing price in one column
megalist_pub['price'] = np.where(megalist_pub['listprice'] == 0.00, megalist_pub['costunitprice'], megalist_pub['listprice'])

#clean column names
megalist_pub.rename(columns={'unitshippingcost': 'shipping_cost', 'maxqtyperorder': 'max_qty', 'shipmentorigin': 'origin'}, inplace = True)

#Arrange columns and drop unnecessary columns

megalist_pub = megalist_pub[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost', 'max_qty', 'origin']]

megalist_pub.head()

### 4.3 Import all_lp and clean it

In [ ]:
all_lp = pd.read_sql(all_lp, prod_db)
all_lp.head()

In [ ]:
all_lp.columns = map(str.lower, all_lp.columns)
all_lp = all_lp.fillna('N/A')
all_lp['isbn'] = all_lp['isbn'].astype(str)
for col in ['isbn','currency']:
    all_lp[col] = all_lp.apply(lambda x: clean_up(x[col]), axis =1)

all_lp.head()

#### 4.3.1 Get biblio with LP

In [ ]:
biblio_lp = pd.merge(all_lp, biblio2, how='left', on = 'isbn' )
biblio_lp.drop(columns = ['pub'], inplace = True)
biblio_lp.head()

### 4.4 Import Sup_by_pub and clean it

In [ ]:
sup_pub = pd.read_excel('../../../../Vendors/.  Vendors Details\supplier_procurement_details.xlsx',  sheet_name='pub')
sup_pub.head()

In [ ]:
sup_pub.columns = map(str.lower, sup_pub.columns)
sup_pub = sup_pub.fillna('N/A')
for col in ['supplier','publisher', 'currency', 'shipmentorigin']:
    sup_pub[col] = sup_pub.apply(lambda x: clean_up(x[col]), axis =1)
sup_pub.head()

#### 4.4.1 Merge Supplier by Publisher with biblio

In [ ]:
sup_pub_biblio = pd.merge(biblio_lp, sup_pub, how='inner', left_on = ['currency', 'publisher'], right_on = ['currency', 'publisher'])
sup_pub_biblio.head()

In [ ]:

sup_pub_biblio.rename(columns={'publisher': 'pub','shipping cost': 'shipping_cost', 'max quantity': 'max_qty', 'shipmentorigin': 'origin', 'shipping cost sea' : 'sea_shipping_cost'}, inplace = True)

sup_pub_biblio.head()

#### 4.5 Import short_disc and clean it

In [ ]:
sp_disc = pd.read_excel("../../../../Vendors/. Short Discounted/all_short_discount_list.xlsx")
sp_disc.head()

In [ ]:
sp_disc.columns = map(str.lower, sp_disc.columns)
sp_disc = sp_disc.fillna('N/A')
sp_disc['isbn'] = sp_disc['isbn'].astype(str)
for col in ['supplier', 'isbn']:
    sp_disc[col] = sp_disc.apply(lambda x: clean_up(x[col]), axis =1)
sp_disc.head()

#### 4.5.1 Merge sup_pub_biblio by sp_disc with biblio

In [ ]:
sup_biblio_disc = pd.merge(sup_pub_biblio, sp_disc, how='left', on = ['supplier', 'isbn'])
sup_biblio_disc.head()

In [ ]:
sup_biblio_disc['discount'] = np.where(sup_biblio_disc['discount_y'].isna(), sup_biblio_disc['discount_x'],sup_biblio_disc['discount_y'])


#Drop unneccessary Columns

sup_biblio_disc.head()

In [ ]:
sup_biblio_disc = sup_biblio_disc[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost', 'sea_shipping_cost', 'max_qty', 'origin']]
sup_biblio_disc.head()

### 4.6 Create table for WC

In [ ]:
can_sup = biblio_lp[biblio_lp['currency'] == 'CND']
can_sup.columns = map(str.lower, can_sup.columns)
can_sup['supplier'] = 'WC'
can_sup['discount'] = np.where(can_sup['price']<150, 0.1, np.where((can_sup['price'] >=150) & (can_sup['price']<200),0.12,0.14))
can_sup['max_qty'] = 10000
can_sup['origin'] = 'CAN'
can_sup['shipping_cost'] = 2
can_sup.rename(columns={'publisher': 'pub'}, inplace = True)
can_sup['currency'] = 'CAD'
can_sup['sea_shipping_cost'] = 2

can_sup = can_sup[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost' , 'max_qty', 'origin']]

can_sup.head()

## 5. Append SupplierMegaLis, PublisherSuppliers and WesterCampus

In [ ]:
all_sup = megalist_pub.append([sup_biblio_disc, can_sup], ignore_index= True)

all_sup_biblio = pd.merge(all_sup,biblio[['isbn', 'publisher']] , how='left', on = 'isbn')

all_sup_biblio['pub'] = np.where(all_sup_biblio['pub'].isna(), all_sup_biblio['publisher'], all_sup_biblio['pub'])

all_sup = all_sup_biblio[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost' , 'max_qty', 'origin']]
all_sup['sea_shipping_cost'] = np.where( (all_sup['origin'] == 'US') | (all_sup['origin'] == 'CAN') , all_sup['shipping_cost'], all_sup['sea_shipping_cost'])
all_sup.isna().sum()

## 6. Add Fees

In [ ]:
#Invoice Fee
all_sup['invoice_fee'] = np.where(     # IF
    all_sup['supplier'].str.contains('ALEK'), 0.005,   #Condition and True
    np.where(  # IF False
        (all_sup['supplier'].str.contains('LAURENTIU')) | (all_sup['supplier'].str.contains('SENAD')),0.02,   #Condition and True
        np.where(  #IF FALSE
            (all_sup['supplier'].str.contains('ARMANDO')) | (all_sup['supplier'].str.contains('FELIPE')),0.01, #Condition and True
            np.where(   #IF FALSE
                (all_sup['supplier'] == 'COINFO'), -1/11, #Condition and True
                0)   # FALSE
                )
                )
                )
# lp_fee

all_sup['lp_fee'] = np.where(
    (all_sup['supplier'].str.contains('BILLSON') & (all_sup['discount'] == 0 )), 0.03,0)

all_sup.head()

## 7. import and Merge exchage rate with all suppliers

In [ ]:
exch_df = pd.read_sql(exchange_rate,proc_db)
exch_df.rename(columns={'name': 'currency'}, inplace = True)
all_sup_exc = pd.merge(all_sup, exch_df, how='left', on = 'currency')
all_sup_exc.head()

## 8. Cost Before exchange rate and landed cost

In [ ]:
all_sup_exc['inv_price_before_exc'] = all_sup_exc['price']*(1-all_sup_exc['discount']+all_sup_exc['lp_fee']*(1+all_sup_exc['invoice_fee']))
all_sup_exc['regular_landed_cost'] = all_sup_exc['inv_price_before_exc'] * all_sup_exc['rate'] + all_sup_exc['shipping_cost']
all_sup_exc['sea_landed_cost'] = all_sup_exc['inv_price_before_exc'] * all_sup_exc['rate'] + all_sup_exc['sea_shipping_cost']
all_sup_exc.head()

## 9. Restrictions

In [ ]:
rest = pd.read_excel("../../../../Vendors/. Restrictions\All Restricted List.xlsx")
rest.columns = map(str.lower, rest.columns)
rest.rename(columns={'isbn13': 'isbn'}, inplace = True)
rest['isbn'] = rest['isbn'].astype(str)
for col in ['isbn', 'vendor']:
    rest[col] = rest.apply(lambda x: clean_up(x[col]), axis =1)

rest = rest[['isbn','vendor']]
rest.head()

In [ ]:
# restricted to all suppliers
risky = rest[rest['vendor'] == 'RISKY']
all_sup_exc = all_sup_exc[~all_sup_exc['isbn'].isin(list(risky['isbn']))]

In [ ]:
# restriction per supplier
all_rest = pd.merge(all_sup_exc, rest, how='left', left_on = ['isbn','supplier'], right_on = ['isbn', 'vendor'] )
all_rest[~(all_rest['vendor'].isna())].head()

## 10. Getting all posible orders for cost_analysis

In [ ]:
all_rest.drop(
    all_rest[
        (all_rest['supplier'] == all_rest['vendor'])
        ].index, inplace = True
)
all_rest.drop(columns=['vendor'], inplace = True)
all_rest = all_rest.sort_values(by=['regular_landed_cost'])
all_rest.head()

In [ ]:
all_rest = all_rest.fillna('N/A')
all_rest.head()

## Extra: Getting suppliers from UK, CAN, US

In [ ]:
first_filter = all_rest[
    (all_rest['origin'] == 'UK')
    | (all_rest['origin'] == 'US')
    | (all_rest['origin'] == 'CAN')
    ]

first_filter.head()

In [ ]:
isbn_list = ['9780134610993',
'9780803677074',
'9781284178418',
'9780803669260',
'9780134752556',
'9780323639088',
'9781492571186',
'9781138683143',
'9780789760500',
'9781496396273',
'9780803677111',
'9781544355658',
'9780803661134',
'9781544331430',
'9780803676787',
'9781544325040',
'9781138088870',
'9780323611077',
'9781506379616',
'9780803675797',
'9780803679917',
'9780803690141',
'9780803674882',
'9780323597791',
'9780803676459',
'9781719640022',
'9780803694736',
'9781319216801',
'9781319216801',
'9781319184568',
'9780134641690',
'9780133953145',
'9780134475585',
'9781118334324',
'9781285740621',
'9781285741550',
'9781118875865',
'9780134169040',
'9781305965720',
'9781111344191',
'9780393640342',
'9780134112107',
'9781319059811',
'9780132273589',
'9780134113593',
'9780077862510',
'9780321962584',
'9781133591887',
'9780323312073',
'9780073384429',
'9781133586845',
'9780321949912',
'9780803658783',
'9780763763381',
'9780534602710',
'9780073385372',
'9780763756093',
'9781584265306',
'9781584265320',
'9781584265290',
'9780199591152',
'9781416406914',
'9780323479820',
'9780078026881',
'9780323402101',
'9780803661158',
'9781138819078',
'9781259677588',
'9781260494198',
'9781138229778',
'9780073513942',
'9781405184588',
'9780849342707',
'9780060786830',
'9780316113502',
'9780465026425',
'9780465052714',
'9780465026425',
'9780465062980',
'9780446404662',
'9780812972764',
'9781599637594',
'9780688184742',
'9780306810121'
]

In [ ]:
uk_us_can = first_filter[first_filter['isbn'].isin(isbn_list)]
uk_us_can.to_csv('retail/uk_us_can.csv', index= False)
# all_suppliers = all_rest[all_rest['isbn'].isin(isbn_list)]
# all_suppliers.to_csv('retail/all_suppliers.csv', index= False)
# elsevier = all_rest[all_rest['pub'].str.contains('ELSEVIER')]
# elsevier.to_csv('retail/elsevier.csv', index= False)
elsevier_cengage = all_rest[all_rest['pub'].str.contains('CENG') | all_rest['pub'].str.contains('ELSE')]
# elsevier_cengage.drop_duplicates(subset ="isbn", keep = False, inplace = True) 
elsevier_cengage.to_csv('retail/elsevier_cengage.csv', index= False)


In [ ]:
# all_rest.to_csv('important_files/python/ca_results.csv', index= False)


